# Set Up

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import folium
import random
import math
%matplotlib inline

from os import path, getcwd
from glob import glob

code_dir = getcwd()
data_dir = path.expanduser('~/data/workshop-content18/3-snc/data/')

ais_pathnames = glob(data_dir + '*.txt')
ais_basenames = [path.basename(pn) for pn in ais_pathnames]

delta_cur_basename = next(bn for bn in ais_basenames if 'Deltaport_Current' in bn)
delta_his_basename = next(bn for bn in ais_basenames if 'Deltaport_History' in bn)

max_rows=int(3e6)
delta_cur = pd.read_csv(
    data_dir + delta_cur_basename, sep='\t', nrows=max_rows, low_memory=False,
    parse_dates=['ReceivedTime'])

In [ ]:
if delta_cur.isna().iloc[-1,:].sum() > 10:
    delta_cur = delta_cur.iloc[:-1, :]

In [ ]:
delta_cur.UserID = delta_cur.UserID.astype('category')

In [ ]:
delta_cur = delta_cur.loc[delta_cur.Latitude < 80]

In [ ]:
nPoints_by_UserID = pd.value_counts(delta_cur.UserID)

In [ ]:
nPoints_threshold = 1000
userID_allowed = nPoints_by_UserID.index[nPoints_by_UserID > nPoints_threshold]
userID_allowed.values.remove_unused_categories(inplace=True)

In [ ]:
delta_cur = delta_cur.loc[delta_cur.UserID.apply(lambda x: x in userID_allowed.values)]

In [ ]:
ReceivedDate = delta_cur.ReceivedTime.dt.date
ReceivedDate_ValueCounts = pd.value_counts(ReceivedDate)
ReceivedDate_ValueCounts.to_frame()

In [ ]:
delta_cur['ReceivedDay'] = ReceivedDate.apply(lambda x: x.day)

In [ ]:
sns.set(style="whitegrid", color_codes=True)

In [ ]:
df_grp = delta_cur.groupby(['ReceivedDay', 'UserID'])

Let's plot records of ships just on the 22nd

In [ ]:
delta_cur_22 = delta_cur.loc[delta_cur.ReceivedDay == 22, :]

# One Ship

In [ ]:
UserID_vc = pd.value_counts(delta_cur.UserID)
#i = random.randint(1,int(UserID_vc.size)) # 117
i = 72 # 112 #95 # 59
ship = (delta_cur.loc[delta_cur.UserID == UserID_vc.index[i]])

xedges = np.load('xedges.dat')
yedges = np.load('yedges.dat')
H = np.load('FS.dat') # feasible set

x = list(ship.Longitude)
y = list(ship.Latitude)

fig = plt.figure(figsize=(15, 15), dpi= 80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, xlabel="Longitude", ylabel="Latitude", title="Feasible Set Of Ships")
plt.imshow(H, interpolation='nearest', origin='low',
           extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]],
           cmap=mpl.cm.Blues)

plt.scatter(x,y, marker='.', color='r', s=5)

In [ ]:
ps = list(zip(x,y)) # all points
butfirst = ps[1:] # all points but first
butlast = ps[:-1] # all points but last
lines = list(zip(butlast,butfirst)) # all directed line segments (ship goes from point one to point two)

In [ ]:
def length(line):
    return math.sqrt((line[1][0] - line[0][0]) ** 2 + (line[1][0] - line[0][0]) ** 2)


In [ ]:
plt.figure(figsize=(15, 5), dpi= 80, facecolor='w', edgecolor='k')
plt.plot([length(x) for x in lines])
plt.ylabel('length (distance units)')
plt.xlabel('Line interval number')
plt.title('Length of line intervals')

# Improving Path

In [ ]:
MAX_LINE_LENGTH = 0.001

# Line -> [Line]
def generateLines(line):
    if length(line) < MAX_LINE_LENGTH:
        return [line]
    else:
        center = ((line[1][0] + line[0][0])/2, (line[1][1] + line[0][1])/2)
        
        line1 = (line[0], center)
        line2 = (center, line[1])
        
        out = []
        out.extend(generateLines(line1))
        out.extend(generateLines(line2))
        return out

lines_improved = []
for line in lines:
    if length(line) > MAX_LINE_LENGTH:
        lines_improved.extend(generateLines(line)) # replace line by a sequence of line segments
    else:
        lines_improved.append(line)

In [ ]:
print(len(lines), len(lines_improved))

In [ ]:
plt.figure(figsize=(15, 5), dpi= 80, facecolor='w', edgecolor='k')
plt.plot([length(x) for x in lines_improved])
plt.ylabel('length (distance units)')
plt.xlabel('Line interval number')
plt.title('Length of Improved Line Intervals')

In [ ]:
points_x2 = list(zip(*lines_improved))
points = points_x2[0]
l = list(zip(*points))
x = l[0]
y = l[1]

In [ ]:
fig = plt.figure(figsize=(15, 15), dpi= 80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, xlabel="Longitude", ylabel="Latitude", title="Feasible Set Of Ships")
plt.imshow(H, interpolation='nearest', origin='low',
           extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]],
           cmap=mpl.cm.Blues)

plt.scatter(x,y, marker='.', color='r', s=2)

# Better Path Estimation: Project Onto Feasible Set

In [ ]:
random.random() - 0.5

In [ ]:
MAX_LINE_LENGTH = 0.001

##### Code:
FS = H
STEP = 0.01

NUM = 1000

START_X = -124
END_X = -122

START_Y = 48.2
END_Y = 49.7

dx = (END_X - START_X)/NUM
dy = (END_Y - START_Y)/NUM

# Point -> (Int, Int) or False
# get i,j indices in the feasible set from coordinates
def getIJ(point):
    if point[0] < START_X or point[0] > END_X:
        return False
    if point[1] < START_Y or point[1] > END_Y:
        return False
    
    return (int((point[0] - START_X)/dx), int((point[1] - START_Y)/dy))

# Point -> Boolean
# produce true if point is in the feasible set
def isFeasible(point):
    result = getIJ(point)
    if result:
        return 1 == FS[result[0], result[1]]
    else:
        return False

# Point -> Point
# Project center onto a feasible set using provided line
def adjustCenter(center, line):
    ship_dir = (line[1][0] - line[0][0], line[1][1] - line[0][0])
    ship_dir_mag = math.sqrt(ship_dir[0] ** 2 + ship_dir[1] ** 2)
    ship_dir_norm = (ship_dir[0]/ship_dir_mag, ship_dir[1]/ship_dir_mag)
    normal = (-ship_dir_norm[1], ship_dir_norm[0])
    
    while not isFeasible(center):
        center = (center[0] + STEP * normal[0],
                  center[1] + STEP * normal[1])
    #print("Adjusted!")
    return center
    
    '''
    Recursive version:
    
    if isFeasible(center):
        return center
    else:
        ship_dir = (line[1][0] - line[0][0], line[1][1] - line[0][0])
        ship_dir_mag = math.sqrt(ship_dir[0] ** 2 + ship_dir[1] ** 2)
        ship_dir_norm = (ship_dir[0]/ship_dir_mag, ship_dir[1]/ship_dir_mag)
        normal = (-ship_dir_norm[1], ship_dir_norm[0])
        center = (center[0] + STEP * normal[0],
                  center[1] + STEP * normal[1])
        
        return adjustCenter(center,line)
    '''

# Line -> [Line]
# Generate missing line segments using feasible set model
def generateLines(line, level=0):
    if length(line) < MAX_LINE_LENGTH:
        return [line]
    elif level > 2000:
        return [line]
    else:
        center = ((line[1][0] + line[0][0])/2, (line[1][1] + line[0][1])/2)
        center = adjustCenter(center, line)
        
        line1 = (line[0], center)
        line2 = (center, line[1])
        
        out = []
        out.extend(generateLines(line1, level=level+1))
        out.extend(generateLines(line2, level=level+1))
        return out

lines_improved = []
for line in lines:
    if length(line) > MAX_LINE_LENGTH:
        lines_improved.extend(generateLines(line)) # replace line by a sequence of line segments
    else:
        lines_improved.append(line)

print('Input number: ',len(lines), '\nOutput number: ', len(lines_improved))

In [ ]:
plt.figure(figsize=(15, 5), dpi= 80, facecolor='w', edgecolor='k')
plt.plot([length(x) for x in lines_improved])
plt.ylabel('length (distance units)')
plt.xlabel('Line interval number')
plt.title('Length of Improved Line Intervals')

In [ ]:
points_x2 = list(zip(*lines_improved))
points = points_x2[0]
l = list(zip(*points))
x = l[0]
y = l[1]

In [ ]:
fig = plt.figure(figsize=(15, 15), dpi= 80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, xlabel="Longitude", ylabel="Latitude", title="Feasible Set Of Ships")
plt.imshow(H, interpolation='nearest', origin='low',
           extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]],
           cmap=mpl.cm.Blues)

plt.scatter(x,y, marker='.', color='r', s=5)